#### Lesson 4: Tool Use and Conversational Tic-Tac-Toe

This lesson will be slightly different from the Lesson 4 in python which uses Chess as an example. This is because the chess library is not available in dotnet, therefore, we will use Tic-Tac-Toe instead.

#### Setup

In [1]:
#r "nuget:AutoGen,0.2.0"
#load "util.csx"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using System.Threading;
using AutoGen.SemanticKernel;
using Microsoft.SemanticKernel;
using System.ComponentModel;
using System.Text;
using OpenAI;
using OpenAI.Chat;

var chatClient = ChatClientProvider.Create("gpt-4o-mini");

Installed Packages AutoGen, 0.2.0

Create TicTacToe class, which helps host the game. The class should have the following methods:
- GetLegalMoves: returns a list of legal moves for the current player
- MakeMove: makes a move for the current player
- DisplayBoard: displays the current board
- CheckWin: checks if the current player has won
- Reset: resets the board

In [2]:
public class TicTacToe
{
    public int[,] board = new int[3, 3];

    public TicTacToe(int[,] board)
    {
        if (board.GetLength(0) != 3 || board.GetLength(1) != 3)
        {
            throw new ArgumentException("The board should be 3x3.");
        }

        this.board = board;
    }

    /// <summary>
    /// Get all legal moves on the board.
    /// </summary>
    [KernelFunction]
    [Description("Get all legal moves on the board.")]
    public async Task<string> GetLegalMoves()
    {
        var legalMoves = new List<int[]>();
        for (var i = 0; i < 3; i++)
        {
            for (var j = 0; j < 3; j++)
            {
                if (board[i, j] == 0)
                {
                    legalMoves.Add([i, j]);
                }
            }
        }

        var sb = new StringBuilder();
        sb.AppendLine("Legal moves:");
        foreach (var move in legalMoves)
        {
            sb.AppendLine($"({move[0]}, {move[1]})");
        }

        return sb.ToString();
    }

    /// <summary>
    /// Display the current board.
    /// </summary>
    [KernelFunction]
    [Description("Display the current board.")]
    public Task<string> DisplayBoard()
    {
        var sb = new StringBuilder();
        sb.AppendLine("Current board:");
        var charMap = new Dictionary<int, string>
        {
            { 0, "0" },
            { 1, "X" },
            { 2, "O" },
        };
        for (var i = 0; i < 3; i++)
        {
            sb.AppendLine(string.Join(" | ", charMap[board[i, 0]], charMap[board[i, 1]], charMap[board[i, 2]]));
        }

        return Task.FromResult(sb.ToString());
    }

    /// <summary>
    /// Make a move on the board.
    /// </summary>
    /// <param name="player">The player making the move (1 for X, 2 for O).</param>
    /// <param name="row">The row to make the move.Must be between 0 and 2.</param>
    /// <param name="col">The column to make the move.Must be between 0 and 2.</param>
    /// <exception cref="ArgumentException"></exception>
    [KernelFunction]
    [Description("Make a move on the board.")]
    public async Task<string> MakeMove(
        [Description("The player making the move (1 for X, 2 for O).")]
        int player,
        [Description("The row to make the move. Must be between 0 and 2.")]
        int row,
        [Description("The column to make the move. Must be between 0 and 2.")]
        int col)
    {
        if (board[row, col] != 0)
        {
            return $"Invalid move. The cell ({row}, {col}) is already occupied.";
        }

        if (player != 1 && player != 2)
        {
            return "Invalid player. Player must be 1 or 2.";
        }

        board[row, col] = player;

        var sb = new StringBuilder();
        sb.AppendLine($"Player {player} made a move at ({row}, {col}).");

        return sb.ToString();
    }

    public bool CheckWin(int player)
    {
        // Check rows
        for (var i = 0; i < 3; i++)
        {
            if (board[i, 0] == player && board[i, 1] == player && board[i, 2] == player)
            {
                return true;
            }
        }

        // Check columns
        for (var i = 0; i < 3; i++)
        {
            if (board[0, i] == player && board[1, i] == player && board[2, i] == player)
            {
                return true;
            }
        }

        // Check diagonals
        if (board[0, 0] == player && board[1, 1] == player && board[2, 2] == player)
        {
            return true;
        }

        if (board[0, 2] == player && board[1, 1] == player && board[2, 0] == player)
        {
            return true;
        }

        return false;
    }

    public void Reset()
    {
        board = new int[3, 3];
    }
}

Initialize the chess board [3, 3] and define the tool middleware
- 0: empty, 1: X, 2: O

In [3]:
var board = new TicTacToe(new int[3, 3]);

var kernel = new Kernel();
var boardToolPlugin = kernel.ImportPluginFromObject(board);
var boardToolPluginMiddleware = new KernelPluginMiddleware(kernel, boardToolPlugin);

Create NestMiddleware for nest check with board

In [4]:
public class NestMiddleware : IMiddleware
{
    public NestMiddleware()
    {
    }

    public string? Name => nameof(NestMiddleware);

    public async Task<IMessage> InvokeAsync(MiddlewareContext context, IAgent agent, CancellationToken cancellationToken = default)
    {
        // check status
        var checkStatusMessage = new TextMessage(Role.User, "check status");
        var status = await agent.SendAsync(chatHistory: [checkStatusMessage]);

        // get legal moves
        var legalMoves = await agent.SendAsync("get legal moves", chatHistory: [checkStatusMessage, status]);

        // make move
        var move = await agent.SendAsync("make move", chatHistory: [status, legalMoves]);

        return move;
    }
}

Create agents

You will create the player agents for the tic-tac-toe game.

In [5]:
var nestMiddleware = new NestMiddleware();
var playerX = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Player_X",
    systemMessage: """
    You are Player X. You are playing Tic-Tac-Toe against Player O.
    You can make a move by providing the row and column number.
    The board is 3x3, and the row and column number should be between 0 and 2.
    First check the status, then get legal moves, and finally make a move.
    """)
    .RegisterMessageConnector()
    .RegisterMiddleware(boardToolPluginMiddleware)
    .RegisterPrintMessage()
    .RegisterMiddleware(nestMiddleware);

var playerO = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Player_O",
    systemMessage: """
    You are Player O. You are playing Tic-Tac-Toe against Player X.
    You can make a move by providing the row and column number.
    The board is 3x3, and the row and column number should be between 0 and 2.
    First check the status, then get legal moves, and finally make a move.
    """)
    .RegisterMessageConnector()
    .RegisterMiddleware(boardToolPluginMiddleware)
    .RegisterPrintMessage()
    .RegisterMiddleware(nestMiddleware);

Start the game!

The max round for a tic-tac-toe game is 9. If no one wins after 9 rounds, the game is a draw.

In [6]:
board.Reset();
// Start the game
var conversationHistory = new List<IMessage>()
{
    new TextMessage(Role.Assistant, "You start first", from: playerX.Name),
};

await foreach(var msg in playerX.SendAsync(receiver: playerO, chatHistory: conversationHistory, maxRound: 9))
{
    conversationHistory.Add(msg);

    // break if anyone wins
    if (board.CheckWin(1))
    {
        Console.WriteLine("Player X wins!");

        break;
    }
    else if (board.CheckWin(2))
    {
        Console.WriteLine("Player O wins!");

        break;
    }

    // print the board
    var displayBoard = await board.DisplayBoard();
    Console.WriteLine(displayBoard);
}

AggregateMessage from Player_O
--------------------
ToolCallMessage:
ToolCallMessage from Player_O
--------------------
- DisplayBoard: {}
--------------------

ToolCallResultMessage:
ToolCallResultMessage from Player_O
--------------------
- DisplayBoard: Current board:
0 | 0 | 0
0 | 0 | 0
0 | 0 | 0

--------------------

--------------------

AggregateMessage from Player_O
--------------------
ToolCallMessage:
ToolCallMessage from Player_O
--------------------
- GetLegalMoves: {}
--------------------

ToolCallResultMessage:
ToolCallResultMessage from Player_O
--------------------
- GetLegalMoves: Legal moves:
(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)
(2, 0)
(2, 1)
(2, 2)

--------------------

--------------------

AggregateMessage from Player_O
--------------------
ToolCallMessage:
ToolCallMessage from Player_O
--------------------
- MakeMove: {"player":2,"row":1,"col":1}
--------------------

ToolCallResultMessage:
ToolCallResultMessage from Player_O
--------------------
- MakeMove

Check the result of the game

In [7]:
if (board.CheckWin(1))
{
    Console.WriteLine("Player X wins!");
}
else if (board.CheckWin(2))
{
    Console.WriteLine("Player O wins!");
}
else
{
    Console.WriteLine("It's a draw!");
}

Player O wins!
